In [ ]:
import io
import sys
import json
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def get_embedding(text):
    # The model.encode() method already returns a list of floats
    return model.encode(text, convert_to_tensor=False).tolist()

In [ ]:
# Read JSON file from path
with open("../../../data/books_data_reviews_sample.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# counter
i = 0

# get time
import time
start_time = time.time()

# Update each document in the JSON data
for document in data:
    print("Processing book: ", i)
    # Extract fields with checks for None
    book_title = document.get("book_title", "") or ""
    description = document.get("description", "") or ""
    authors = document.get("authors", "") or ""
    categories = document.get("categories", "") or ""

    # Combine relevant text fields
    combined_text = book_title + " " + description + " " + authors + " " + categories

    # Add additional fields if needed
    # For example, you might want to include publisher and publishedYear

    

    # Process reviews if they exist
    reviews = document.get("reviews", [])
    if len(reviews) > 0:
        for review in reviews:
            summary = review.get("summary", "") or ""
            text = review.get("text", "") or ""

            # Combine review summary and text
            review_text = summary + " " + text
            combined_text += " " + review_text
            # Create an embedding for the review text
            # review_embedding = get_embedding(review_text)

            # Add the review embedding to the review dictionary
            # review["vector"] = review_embedding

    # Create an embedding for the combined text
    document["vector"] = get_embedding(combined_text)
    i += 1

    print("Time elapsed: " + str(time.time() - start_time) + ". Estimated time remaining: " + str((time.time() - start_time) / i * (len(data) - i)/60) + " minutes")

In [ ]:
# Save the modified data back to the JSON file
with open('../../../data/books_data_reviews_sample_embeddings_new.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)